## Best Practice to Calculate and Interpret Feature Importance for RF model

**Motivation**:         
Having a model that accurately predicts outcome is great, but in many cases we also need to interpret our model. Let's say we just built a model to predict customers' renewal rate, in additional to knowing who will renew, it's perhaps equally important to understand which variables are importance in determining the renewal forecast to help improve our product and service.
  
People often use feature importances to interpret the model and most statistical models offer default calcuation in Sciki-learn package to make our lives easier. However, oftentime we can't trust the default feature importance. 

In this article, we are going to use the famous Titanic data and a random forest model to illustrate:
- Why you need a **robust** model and **permutation importance** score to properly calculate feature importance.
- Why you need to understand **features correlation** to properly interpret the feature importances.

The practice described in this article can be generalized to other models as well. 

In [4]:
#!pip install rfpimp
import linkedin.darwinfilemanager
import pandas as pd
import numpy as np
import datetime
import warnings
import seaborn as sns
from matplotlib.pyplot import figure
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, recall_score
from distutils.version import LooseVersion
import sklearn
from sklearn.base import clone
from rfpimp import *
from sklearn.preprocessing import OrdinalEncoder
from sklearn.inspection import permutation_importance
warnings.filterwarnings("ignore")

### Best Practice to Calculate Feature Importances

#### Trouble with Default Feature Importance

We are going to use an example to show the **problem with the default impurity-based feature importance** provided in Sciki-learn for Random Forest model. The mechanism of default feature importance is calculated based on mean decrease in impurity (or gini importance), it measure how effective the features is at reducting uncertainty. See [this great article](https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3) to illustrate the math behind the feature importance calculation.

We download the famous Titanic dataset from [Kaggle](https://www.kaggle.com/c/titanic/data). The data set has passenger information for 1309 passgaes on Titanic and whether they ended up surviving the disaster. Here is brief description of columns included in the data.

| Variable      | Definition | Key|
| ----------- | ----------- |-------|
| survival      | Our response variable - Survival or not     |0 = No, 1 = Yes|
| Pclass   | Ticket class |1 = 1st, 2 = 2nd, 3 = 3rd|
|sex|Sex||
|Age|Age in years||
|sibsp|# of siblings / spouses aboard the Titanic||
|parch|# of parents / children aboard the Titanic||
|ticket|Ticket number||
|fare|Passenger fare||
|cabin|Cabin number||
|embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton|

First, we separate the data into predictor set and response set. In the predictor set, we added two random variable random_cat and random_num. Since they are randomly generated, neither of them should show up as important features and they should roughly have similar ranks of feature importance.

In [5]:
df = pd.read_csv('darwin://xianli-adhoc/titanic.csv')
df = df.dropna(subset=['survived'])
X = df.drop("survived", axis = 1)
y = df['survived']
rng = np.random.RandomState(seed=42)
X["random_cat"] = rng.randint(3, size=X.shape[0])
X["random_num"] = rng.randn(X.shape[0])

Second, we did some simple cleaning and transformation on the data. This is not the focus of this article.

In [6]:
categorical_columns = ["pclass", "sex", "embarked", "random_cat"]
numerical_columns = ["age", "sibsp", "parch", "fare", "random_num"]

#did a simple imputation to fill missing categorical var with mode. The only var missing is embarked with 2 missing values 
X['embarked'] = X['embarked'].fillna(X['embarked'].mode()[0])
enc = OrdinalEncoder()
X[categorical_columns] = enc.fit_transform(X[categorical_columns] )

#imputed numerical var missing values with mean 
X[numerical_columns] = X[numerical_columns].apply(lambda x: x.fillna(np.mean(x)))

X = X[categorical_columns + numerical_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

Third, we build a random forest model.

In [7]:
rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 1,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

In [8]:
print(f"RF train accuracy: {rf.score(X_train, y_train):.3f}")
print(f"RF test accuracy: {rf.score(X_test, y_test):.3f}")

Looks like the model is overfitting on training data but still have decent performance on testing set. Let's use this model for now to illustrate some pitfalls of default feature importance calculation.

In [9]:
feat_importances = pd.Series(rf.feature_importances_, index = X_train.columns).sort_values(ascending = True)
feat_importances.plot(kind = 'barh')

From the default feature importance, we notice that:
- The random_num has a larger importance score in comparison with random_cat, which confirms that **impurity-based importances are biased towards high cardinality and numerical features**.
- Non-predictive random_num variable is ranked as one of the most importance features, which is very suspicious. This reflects the **default features importance is not accurate when you have an overfitting model**. When a model overfits, it's picking up too much noise from training set that its ability to make generalized prediction on the test has been reduced. When this happens,the feature importance is not reliable as it's calculated based on training set. More generally, **it only makes sense to look at feature importance when you have a model that can decently predict.**


#### Permutation Importances to Rescue

Then how do we appropriately calculate the importance of features. One way is to use permutation importances scores. It's calculated with following steps: 
1. Train a baseline model and record the score (we use accuracy in this example) on the validation set. 
2. Re-shuffle values for one feature, use the model to predict again and calculate score on validation set. The feature importance is the difference between the baseline in 1 and the permutation score in 2.
3. Repeat the process for all features.

Here we leveraged `permutation_importance` function added to `Scikit-learn` package in 2019. When calling the function, we set the n_repeats = 20 which meant for each variable, we randomly shuffle 20 times and calculate the decrease in accuracy 20 times to create the boxplot.

In [10]:
from sklearn.inspection import permutation_importance
#calculate permutation importance for test data 
result_test = permutation_importance(
    rf, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_test = result_test.importances_mean.argsort()
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx_test].T,
    columns=X.columns[sorted_importances_idx_test],
)

#calculate permutation importance for training data 
result_train = permutation_importance(
    rf, X_train, y_train, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_train = result_train.importances_mean.argsort()
importances_train = pd.DataFrame(
    result_train.importances[sorted_importances_idx_train].T,
    columns=X.columns[sorted_importances_idx_train],
)

f, axs = plt.subplots(1,2,figsize=(15,5))

importances_test.plot.box(vert=False, whis=10, ax = axs[0])
axs[0].set_title("Permutation Importances (test set)")
axs[0].axvline(x=0, color="k", linestyle="--")
axs[0].set_xlabel("Decrease in accuracy score")
axs[0].figure.tight_layout()

importances_train.plot.box(vert=False, whis=10, ax = axs[1])
axs[1].set_title("Permutation Importances (train set)")
axs[1].axvline(x=0, color="k", linestyle="--")
axs[1].set_xlabel("Decrease in accuracy score")
axs[1].figure.tight_layout()

We see that `sex` and `pclass` shows as the most important features and `random_cat` and `random_num` are no longer have high importance scores based on permutation importance on test set. The box plot shows the distribution of decrease in accuracy score in N repeat permutation (N = 20).

Let's also compute permutation imporatnce on training set. This shows that `random_num` and `random_cat` get a significantly higher importance ranking than when computed on the test set and the ranking of features have shifted significantly. As noted before, it's due to the overfit of the model.

You may wonder why Sciki-learn still includes a default calculation for feature importance given it's not accurate. Breiman and Cutler, the inventors of RFs, indicate that this method of “adding up the gini decreases for each individual variable over all trees in the forest gives a fast variable importance that is often very consistent with the permutation importance measure.” So the default is meant to serve as a proxy for permutation importance. However, as Strobl et al pointed out in `Bias in random forest variable importance measures`: Illustrations, sources and a solution that “the variable importance measures of Breiman's original Random Forest method ... are not reliable in situations where potential predictor variables vary in their scale of measurement or their number of categories.”

#### A Robost Model is the Pre-requisite for Accurate Importance Scores

Let's apply some level of regularization by setting min_samples_leaf = 20 instead of 1.

In [11]:
rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 20,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

In [12]:
print(f"RF train accuracy: {rf.score(X_train, y_train):.3f}")
print(f"RF test accuracy: {rf.score(X_test, y_test):.3f}")

When we adjust the model parameters to avoid overfitting, we notice that accuracy now are very similar.

In [13]:
#calculate permutation importance for test data 
result_test = permutation_importance(
    rf, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_test = result_test.importances_mean.argsort()
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx_test].T,
    columns=X.columns[sorted_importances_idx_test],
)

#calculate permutation importance for training data 
result_train = permutation_importance(
    rf, X_train, y_train, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx_train = result_train.importances_mean.argsort()
importances_train = pd.DataFrame(
    result_train.importances[sorted_importances_idx_train].T,
    columns=X.columns[sorted_importances_idx_train],
)

f, axs = plt.subplots(1,2,figsize=(15,5))

importances_test.plot.box(vert=False, whis=10, ax = axs[0])
axs[0].set_title("Permutation Importances (test set)")
axs[0].axvline(x=0, color="k", linestyle="--")
axs[0].set_xlabel("Decrease in accuracy score")
axs[0].figure.tight_layout()

importances_train.plot.box(vert=False, whis=10, ax = axs[1])
axs[1].set_title("Permutation Importances (train set)")
axs[1].axvline(x=0, color="k", linestyle="--")
axs[1].set_xlabel("Decrease in accuracy score")
axs[1].figure.tight_layout()

After fixing the overfit, the features importance calculated on training and testing set look much similar to each other. This gives us more confidence that a robust model gives more accurate model importance. 

An alternative way to calculate accurate feature importance is **drop-column importance**. It's the most accurate way to calculate feature importance. The idea is to calculate the model performance with all predictors and drop the signal predictor and see the reduction in performance. The more important the feature is, the higher the reduction we see in the model performance. 

Given the high computation cost of drop-column importance (we need to retain the a model for each variable), we generally prefer permutation Importance score. But it's an excellent way to validate our permutation importance. The importance values could be difference between the two strategies, but the order of feature importances should be roughly the same.

In [14]:
def dropcol_importances(rf, X_train, y_train):
    rf_ = clone(rf)
    rf_.random_state = 42
    rf_.fit(X_train, y_train)
    
    #use out of bag error as performance measurement
    baseline = rf_.oob_score_
    imp = []
    for col in X_train.columns:
        X = X_train.drop(col, axis=1)
        rf_ = clone(rf)
        rf_.random_state = 42
        rf_.fit(X, y_train)
        o = rf_.oob_score_
        imp.append(baseline - o)
    imp = np.array(imp)
    I = pd.DataFrame(
            data={'Feature':X_train.columns,
                  'Importance':imp})
    I = I.set_index('Feature')
    I = I.sort_values('Importance', ascending=True)
    return I

In [15]:
imp = dropcol_importances(rf, X, y)

In [16]:
imp.plot(kind = 'barh')

The rankings of the features are similar to permutation features although the magnitude is different.

### Best Practice to interpret feature importances

#### The Challenge of Feature Correlation

After we have a robust model and correctly implement the right strategy to calculate feature importance, we can move forward to the interpretation. Can we argue that one feature is 10X more importance than another? It depends.   

At this stage, correlation is the biggest challenge for us to interpret the feature importance. The assumption we make so far consider each feature individually. If all features are totally independent and not correlated in any way, it would easy to make the interpretation. However, if two or more features are collinear, it will affect the feature importance result.

To illustrate this, let's use an extreme example and duplicate the column sex to retrain the model.

In [17]:
X_train['sex_dedupe'] = X_train['sex'].copy()
X_test['sex_dedupe'] = X_test['sex'].copy()
rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 20,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

In [18]:
print(f"RF train accuracy: {rf.score(X_train, y_train):.3f}")
print(f"RF test accuracy: {rf.score(X_test, y_test):.3f}")

The model performance slightly decreases as we added an feature that didn't add much information.

In [19]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    rf, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_train.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()
#show the original graph 

We see now the importance attributed to sex features are distributed between two duplicates sex columns. What happen if we add some noises to the duplicate column? Let's try adding a random noise ranging from 0-1 to the sex.

In [20]:
#Adding noise to the features 
X_train = X_train.drop("sex_dedupe", axis = 1)
X_test = X_test.drop("sex_dedupe", axis = 1)

noise_train = np.random.random(len(X_train['sex']))*1
noise_test = np.random.random(len(X_test['sex']))*1

X_train['sex_noisy'] = X_train['sex'] + noise_train
X_test['sex_noisy'] = X_test['sex'] + noise_test

rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 20,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

result = permutation_importance(
    rf, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_train.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

`sex_noisy` now is the most importance variable. What happen if we increase the magnitude of noise added? Let's increase the random varaible to ranging from 0-3.

In [21]:
#Adding noise to the features 
#Increse the magnitude of noise
noise_train = np.random.random(len(X_train['sex']))*3
noise_test = np.random.random(len(X_test['sex']))*3

X_train['sex_noisy'] = X_train['sex'] + noise_train
X_test['sex_noisy'] = X_test['sex'] + noise_test

rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 20,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

result = permutation_importance(
    rf, X_test, y_test, n_repeats=20, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X_train.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

Now we can see with more noise added, `sex_noisy` now no longer rank as the top predictors and `sex` is back to the top. The conclusion is that permutation importance computed on random forest model **spreads importance across collinear variables**. The amount of sharing appears to be a function of how much noise there in between the two. 

#### Dealing with Collinear Features

Let's take a look at correlation between features. We use `feature_corr_matrix` from rfpimp package, which gives us the Spearman's correlation. The difference between Spearman's correlation and standard Pearson's correlation is that Spearman's correaltion first converts two variables to rank values and then run Peason correlation on ranked values. It doesn't assume a linear relationship between variables.

In [22]:
feature_corr_matrix(X_train)

In [23]:
from rfpimp import plot_corr_heatmap
viz = plot_corr_heatmap(X_train, figsize=(7,5))
viz.view()

`pclass` is highly correlated with `fare`, which is not too surprising as class of cabin depends on how much money you pay for it. It happens quite often in business that we use multiple features that are correlated with each other in the model for prediction. From the previous example, we see that when two or multiple are collinear, the importance computed are shared across collinear var based on the info to noise ratio.

#### Strategy 1: Combine Collinear Feature

One way to tackle this is to combine features that are highly collinear with each other to form a feature family and we can say this feature family together ranked as the X most importance features. To to that, we will use `rfpimp` package that allows us to permutate two variables at one time.

In [29]:
import rfpimp as rfpimp
features = [['pclass','fare'],'sex','embarked','age','sibsp','parch']
X_train = X_train.drop(['sex_noisy','random_num','random_cat'], axis = 1)
X_test = X_test.drop(['sex_noisy','random_num','random_cat'], axis = 1)

rf = RandomForestClassifier(
         n_estimators=100,
         n_jobs=-1,
         min_samples_leaf = 20,
         oob_score=True,
         random_state = 42)
rf.fit(X_train, y_train)

I = rfpimp.importances(rf, X_test, y_test, features=features)
plot_importances(I)

#### Strategy 2: Remove Highly Collinear Variable

If a feature is dependent on other features that means the features can be accurately predicted using all other features as independe variables. Higher the model R^2 is , the more dependent feature is, and the more confidence we can remove the variable without sacrificing on the accuracy.

In [30]:
dependence_matrix = feature_dependence_matrix(X_train,
                          rfrmodel=RandomForestRegressor(n_estimators=50, oob_score=True),
                          rfcmodel=RandomForestClassifier(n_estimators=50, oob_score=True),
                          cat_count=20,
                          zero=0.001,
                          sort_by_dependence=False,
                          n_samples=5000)

In [31]:
dependence_matrix

In [32]:
plot_dependence_heatmap(dependence_matrix)

The first column dependence shows the dependence score and the feature is completely predictable using other other features the value is close to 1, which means it could be dropped without affecting accuracy. In this case, we can probably drop one of the pclass and fare without affecting much accruacy.

### At the End

Once we 1)have a robust model and implement the right strategy to calculate permutation importance and 2)deal with feature correlation, we can start crafting our message to share with stakeholder. 

Going back the common question people ask "Is feature 1 10x more importance feature 2?", you may understand at this moment that we have confidence to make the argument only when all the features are independent or have very low correlation. But in the real world, that's rarely the case. The recommended strategy is to group features to High, Medium and Low impact tier, without focusing too much on the exact magnitude. If we really need to show the relatively comparison between features, try to group collinear features (or drop them) and interpret based on category to make the argument more accurate.

### Reference 

https://explained.ai/rf-importance/    
https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html   
https://github.com/parrt/random-forest-importances   
https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3
https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e